# Imports

In [71]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import random

# Load data

In [72]:
data_directory = '../input/gem-predictive-maintenance-2022'
chosen_dataset = 'FD003'

df_train = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_train.csv'), index_col=0) 
df_test = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_test.csv'), index_col=0) 

# Add the RUL

In the current dataset the RUL is not part of the column.

In the data descrition, it is mentionned that in the training data, the engine runs until failure.

Given the time in cycles, it is easy to retreive the RUL.



In [73]:
def add_rul(g):
    g['RUL'] = max(g['time_in_cycles']) - g['time_in_cycles']
    return g

df_train = df_train.groupby('engine_no').apply(add_rul)
df_test = df_test.groupby('engine_no').apply(add_rul)

In [74]:
df_test

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,RUL
0,1,1,-0.0017,-0.0004,100.0,518.67,641.94,1581.93,1396.93,14.62,...,100.0,39.07,23.4468,NaN,NaN,NaN,NaN,NaN,NaN,232
1,1,2,0.0006,-0.0002,100.0,518.67,642.02,1584.86,1398.90,14.62,...,100.0,39.04,23.4807,NaN,NaN,NaN,NaN,NaN,NaN,231
2,1,3,0.0014,-0.0003,100.0,518.67,641.68,1581.78,1391.92,14.62,...,100.0,39.10,23.4244,NaN,NaN,NaN,NaN,NaN,NaN,230
3,1,4,0.0027,0.0001,100.0,518.67,642.20,1584.53,1395.34,14.62,...,100.0,38.97,23.4782,NaN,NaN,NaN,NaN,NaN,NaN,229
4,1,5,-0.0001,0.0001,100.0,518.67,642.46,1589.03,1395.86,14.62,...,100.0,39.09,23.3950,NaN,NaN,NaN,NaN,NaN,NaN,228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16591,100,243,0.0011,-0.0003,100.0,518.67,643.04,1594.99,1411.28,14.62,...,100.0,39.40,23.4949,NaN,NaN,NaN,NaN,NaN,NaN,4
16592,100,244,-0.0024,0.0002,100.0,518.67,642.47,1591.27,1417.64,14.62,...,100.0,39.42,23.6011,NaN,NaN,NaN,NaN,NaN,NaN,3
16593,100,245,-0.0053,0.0002,100.0,518.67,642.70,1593.81,1412.70,14.62,...,100.0,39.43,23.5482,NaN,NaN,NaN,NaN,NaN,NaN,2
16594,100,246,-0.0006,0.0001,100.0,518.67,642.19,1595.63,1406.58,14.62,...,100.0,39.40,23.6687,NaN,NaN,NaN,NaN,NaN,NaN,1


Import pour le machine learning

In [75]:
import sklearn                       #machine learning functions

from sklearn.neural_network import MLPClassifier      # this function creates the neural network "Multi-Layer Perceptron Classifier"
from sklearn.neural_network import MLPRegressor       # Backpropagation
from sklearn.model_selection import train_test_split  # split train and test sets
from sklearn.metrics import mean_squared_error        # RMSE
from math import sqrt                                 # yes, python needs you load a package for sqrt
from sklearn.metrics import r2_score                  # Coefficient of determination

In [76]:


#Y :
target_column =['RUL']  #this is the outcome/dependent variable Y
#X :
selected_features = [ 'time_in_cycles', 'op_setting_1', 'op_setting_2',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8',
       'sensor_9', 'sensor_11', 'sensor_12', 'sensor_13',
       'sensor_14', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21']


In [77]:
print([df_train])

[       engine_no  time_in_cycles  op_setting_1  op_setting_2  op_setting_3  \
0              1               1       -0.0005        0.0004         100.0   
1              1               2        0.0008       -0.0003         100.0   
2              1               3       -0.0014       -0.0002         100.0   
3              1               4       -0.0020        0.0001         100.0   
4              1               5        0.0016        0.0000         100.0   
...          ...             ...           ...           ...           ...   
24715        100             148       -0.0016       -0.0003         100.0   
24716        100             149        0.0034       -0.0003         100.0   
24717        100             150       -0.0016        0.0004         100.0   
24718        100             151       -0.0023        0.0004         100.0   
24719        100             152        0.0000        0.0003         100.0   

       sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  ...  

Creating the training and test datasets.

In [78]:
x_train = df_train[selected_features].values.astype('int')    # df to numpy.ndarray (tableau)
y_train = df_train[target_column].values.astype('int')

x_test = df_test[selected_features].values.astype('int')
y_test = df_test[target_column].values.astype('int')

print(x_train.shape)
print(y_train)


(24720, 17)
[[258]
 [257]
 [256]
 ...
 [  2]
 [  1]
 [  0]]


TEST

In [79]:
mlp = MLPClassifier(hidden_layer_sizes=(17,17,17), activation='relu', solver='adam')

mlp.fit(x_train,y_train.ravel())

predict_train = mlp.predict(x_train)
predict_test = mlp.predict(x_test)

In [80]:
df_resultat=df_train.assign(RUL_predict=predict_train)
df_resultat_test=df_test.assign(RUL_predict=predict_test)
#print (df_resultat)

In [81]:
test=['engine_no' , 'RUL_predict', 'RUL']
df_final=df_resultat[test]
df_final = df_final.groupby('engine_no').agg({'RUL_predict': 'first'}).reset_index()
print(df_final)
df_final_test = df_resultat_test[test]
df_final_test = df_final_test.groupby('engine_no').agg({'RUL_predict': 'first'}).reset_index()
print(df_final_test)
#df_final.columns = ['engine_no', 'RUL']
#df_final[['engine_no','RUL']].to_csv('submission.csv', index=False)
#df_final


    engine_no  RUL_predict
0           1          143
1           2          143
2           3          143
3           4          143
4           5          143
..        ...          ...
95         96          143
96         97          143
97         98          143
98         99          143
99        100          143

[100 rows x 2 columns]
    engine_no  RUL_predict
0           1          143
1           2          143
2           3          143
3           4          143
4           5          143
..        ...          ...
95         96          143
96         97          143
97         98          143
98         99          143
99        100          143

[100 rows x 2 columns]


In [82]:
df_test_resultat = df_final
df_test_resultat ['RUL'] = df_resultat['RUL']
df_test_resultat ['difference'] = df_test_resultat ['RUL']- df_test_resultat['RUL_predict']
df_test_resultat_test = df_final_test
df_test_resultat_test ['RUL'] = df_resultat_test['RUL']
df_test_resultat_test ['difference'] = df_test_resultat_test ['RUL']- df_test_resultat_test['RUL_predict']
print(df_test_resultat_test)
print(df_test_resultat.describe())
print(df_test_resultat_test.describe())

    engine_no  RUL_predict  RUL  difference
0           1          143  232          89
1           2          143  231          88
2           3          143  230          87
3           4          143  229          86
4           5          143  228          85
..        ...          ...  ...         ...
95         96          143  137          -6
96         97          143  136          -7
97         98          143  135          -8
98         99          143  134          -9
99        100          143  133         -10

[100 rows x 4 columns]
        engine_no  RUL_predict         RUL  difference
count  100.000000        100.0  100.000000  100.000000
mean    50.500000        143.0  208.500000   65.500000
std     29.011492          0.0   29.011492   29.011492
min      1.000000        143.0  159.000000   16.000000
25%     25.750000        143.0  183.750000   40.750000
50%     50.500000        143.0  208.500000   65.500000
75%     75.250000        143.0  233.250000   90.250000
max    1

In [83]:
df_final_test.columns = ['engine_no', 'RUL']
df_final_test[['engine_no','RUL']].to_csv('submission.csv', index=False)
df_final_test

ValueError: Length mismatch: Expected axis has 4 elements, new values have 2 elements

# You work for the next lesson:

By groups of 3 :
- Try to have the best model on FD003
- Prepare a quick presentation (5 min) explaining your data analysis and you model.
- Explain how you would answer to the following issue :
«I need to predict failure on my engine. Considering that each engine does 8 cycles a day on average. If my engine fails, it costs 4m to repair, and 500k of operational impact. If I repair the engine before failure it costs me 1m and depending on removal planning, costs are different. If I plan my maintenance within 5 days it costs me 400k of operational impact. From 5 to 10 days 200k, and above 10 days 100k. Also I know that if I remove an engine before it fails I lose some potential (~3,5k / cycle);»